Случайный лес - ансамблевый алгоритм машинного обучения над деревьями. В отличие от бустингов, деревья в этом алгоритме строятся параллельно и независимо.

In [29]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
import optuna

In [30]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [31]:
df = pd.read_csv('../data/extract/data.csv')

In [32]:
df.head()

,Unnamed: 0,Цена соевого масла_Роттердам__root_mean_square_x,Цена сырого пальмового масла__maximum_x,Цена сырого пальмового масла__root_mean_square_x,"Цена соевого масла_Роттердам__linear_trend__attr_""pvalue""_X1",Цена соевого масла_Аргентина__standard_deviation_X1,Цена пальмового масла_Аргентина__root_mean_square_X1,Цена соевого масла_Аргентина__variance_X1,Цена соевого масла_Бразилия__standard_deviation_X1,Цена соевого масла_Бразилия__variance_X1,...,Цена соевого масла_Бразилия__length_week,Цена соевого масла_Бразилия__mean_week,Цена соевого масла_Бразилия__median_week,Цена соевого масла_Бразилия__sum_values_week,"Цена соевого масла_Аргентина__linear_trend__attr_""slope""_week","Цена соевого масла_Аргентина__linear_trend__attr_""pvalue""_week",Цена соевого масла_Аргентина__minimum_week,Цена сырого пальмового масла__median_week,date,Цена сырого пальмового масла
0,0,897.281784,1485.0,871.823491,3.068082e-52,233.296209,977.98713,54427.121024,235.145601,55293.453658,...,682.0,947.196334,878.0,645987.9,0.108246,0.040672,555.57,765.000,2011-01-17,1215.0
1,1,897.281784,1485.0,871.823491,3.068082e-52,233.296209,977.98713,54427.121024,235.145601,55293.453658,...,682.0,946.779912,873.6,645703.9,0.106434,0.043488,556.01,768.125,2011-01-18,1205.0
2,2,897.281784,1485.0,871.823491,3.068082e-52,233.296209,977.98713,54427.121024,235.145601,55293.453658,...,681.0,947.314831,874.6,645121.4,0.109991,0.038626,547.19,765.000,2011-01-19,1245.0
3,3,897.281784,1485.0,871.823491,3.068082e-52,233.296209,977.98713,54427.121024,235.145601,55293.453658,...,681.0,946.309104,875.7,644436.5,0.101916,0.053841,552.04,767.500,2011-01-20,1245.0
4,4,897.281784,1485.0,871.823491,3.068082e-52,233.296209,977.98713,54427.121024,235.145601,55293.453658,...,681.0,946.632159,873.7,644656.5,0.107420,0.042281,561.74,767.500,2011-01-21,1255.0


In [33]:
X = df.drop(['Цена сырого пальмового масла', 'date'], axis=1)
y = df['Цена сырого пальмового масла']

In [34]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 8, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_float('max_features', .0, 1.),
        'random_state': 42,
        'n_jobs': -1
    }

    tscv = TimeSeriesSplit(n_splits=100)
    mape_arr = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = RandomForestRegressor(**param)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        mape_arr.append(mape(y_test, y_pred))

    return np.mean(mape_arr) + np.std(mape_arr)


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

[I 2024-04-16 22:49:45,281] A new study created in memory with name: no-name-e1251904-fa00-4daf-a3a1-4876ea58ef78
[I 2024-04-16 22:52:57,614] Trial 0 finished with value: 14.472852630031586 and parameters: {'n_estimators': 492, 'max_depth': 46, 'min_samples_split': 18, 'min_samples_leaf': 2, 'max_features': 0.7585267463009866}. Best is trial 0 with value: 14.472852630031586.


In [ ]:
best_params = study.best_params

In [ ]:
best_model = RandomForestRegressor(**best_params)

train = df.set_index('date').loc[:'2024-01-01']
test = df.set_index('date').loc['2024-01-01':]


X_train = train.drop('Цена сырого пальмового масла', axis=1)
y_train = train['Цена сырого пальмового масла']

X_test = test.drop('Цена сырого пальмового масла', axis=1)
y_test = test['Цена сырого пальмового масла']


best_model.fit(X_train, y_train)


y_pred = best_model.predict(X_test)


plt.figure(figsize=(20, 7), dpi=300)
plt.plot(X_test.index, y_test, label='Actual Price', color='blue')
plt.plot(X_test.index, y_pred, label='Predicted Price', color='red')  # Возврат к исходной шкале
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.ylabel('Price')
plt.ylim(600, 950)
plt.title('Actual vs Predicted Price')
plt.legend()
plt.show()

print("MAPE", mape(y_test, y_pred))